# Visualizing Objectives



Principles:

1. Objective chains are directed and acyclic.

2. Each node represents a discrete competency cluster.

3. Each edge represents a dependency of one competency cluster on another.

4. Competency clusters should be as coherent as possible.

2. Edge chains should be independent.  I.e. one should not link to more than one previous node in a chain.  (One may link to both nodes in a forked chain.)
    
    For instance, if 303 depends on 170 and 190, but 190 depends on 170, then only 190 should be linked to 303.


##  Load and clean CSV of nodes

In [1]:
# Load and clean CSV of nodes and descriptions.
from csv import DictReader

nodes = {}
edges = {}
with open('curr-raw.csv', 'r') as rawFile:
    rawData = DictReader(rawFile)
    
    # Build nodes.
    for line in rawData:
        if line["Topic"] in ['', 'Aggregate'] or line["Competency Cluster"] in '': continue
        nodes[line['#']] = f'{line["Topic"]}: {line["Competency Cluster"]}'
        
        if line['Antecedents'].strip() == '': continue
        edges[line['#']] = []
        ants = line['Antecedents'].split(',')
        for ante in ants:
            if line["Antecedents"].strip() == '0': continue
            edges[line['#']].append(ante.strip())
            #print(line['#'], edges[line['#']])

##  Convert to graph representation

In [2]:
import pygraphviz as pgv

G = pgv.AGraph(edges, strict=True, directed=True, rankdir="LR")

G.layout('dot')
G.draw('curr.png')
G.draw('curr.svg')

##  Manually build palette for Hoon School topics

In [10]:
data = '''100
110, 113, 103
115, 120, 125, 175
130, 133, 135
140, 145, 160
150, 155, 165
156, 183
184, 233
163, 180
170, 190, 217'''

colors = {
    0:'#9e0142',
    1:'#d53e4f',
    2:'#f46d43',
    3:'#fdae61',
    4:'#fee08b',
    5:'#e6f598',
    6:'#abdda4',
    7:'#66c2a5',
    8:'#3288bd',
    9:'#5e4fa2',
}
#http://colrd.com/palette/18894/ "Spectral 10" by 
nodes = {}
for idx,row in enumerate(data.split('\n')):
    for val in row.split(','):
        nodes[val.strip()] = colors[idx]

##  Mark SVG nodes by color.

In [11]:
from bs4 import BeautifulSoup
import re

with open('curr.svg') as f:
    svg = f.read()

soup = BeautifulSoup(svg, 'lxml')

svg_tag = soup.find('svg')
svg_grf = svg_tag.find('g')

paths = svg_grf.find_all('g', {'class': 'node'})

for path in paths:
    try:
        title = path.findChild('title')
    except:
        continue
    if title.contents[0] in nodes:
        ellipse = path.findChild('ellipse')
        color = nodes[title.contents[0]]
        ellipse['fill'] = f'{color}'

with open('curr-marked.svg', 'w') as f:
    f.write(str(svg_tag))

##  Build objectives list.

In [12]:
data = '''100
110, 113, 103
115, 120, 125, 175
130, 133, 135
140, 145, 160
150, 155, 165
156, 183
184, 233
163, 180
170, 190, 217'''

lessons = {}
for idx,row in enumerate(data.split('\n')):
    lessons[idx] = []
    for val in row.split(','):
        lessons[idx].append(val.strip())

import yaml
for lesson in lessons:
    print(f'After Lesson {lesson}, you should be able to:\n')
    for node in lessons[lesson]:
        # whatever your local path is
        fname = f'/home/neal/urbit/curriculum/lessons/{node}.yml'
        with open(fname, 'r') as f:
            data = f.read()
        data = yaml.safe_load(data)
        for obj in data['objectives']:
            print('-', obj)
    
    print('\nYou will know the runes:\n')
    for node in lessons[lesson]:
        # whatever your local path is
        fname = f'/home/neal/urbit/curriculum/lessons/{node}.yml'
        with open(fname, 'r') as f:
            data = f.read()
        data = yaml.safe_load(data)
            
        for rune in data['runes']:
            print(f'- `{rune}`')
    print('\n---\n')

After Lesson 0, you should be able to:

- Explain what an Urbit ship is.
- Identify the `.urb/` directory.
- Distinguish a fakeship from a liveship.
- Use the `+ls` generator to show a directory's contents.
- `|mount` and `|commit` a desk.

You will know the runes:


---

After Lesson 1, you should be able to:

- Distinguish nouns, cells, and atoms.
- Apply auras to transform an atom.
- Identify common Hoon molds, such as cells, lists, and tapes.
- Annotate Hoon code with comments.
- Pin a face to the subject.
- Make a decision at a branch point.
- Distinguish loobean from boolean operations.
- Slam a gate (call a function).
- Produce a generator to convert a value between auras.
- Pronounce ASCII characters per standard Hoon developer practice.

You will know the runes:

- `::`
- `%-`
- `=/`
- `?:`
- `^-`
- `~&`
- ``

---

After Lesson 2, you should be able to:

- Identify current known irregular syntax.
- Convert between regular and irregular forms of runes to date.
- Identify a mold

In [135]:
lessons

{0: ['100'],
 1: ['110', '113', '103'],
 2: ['115', '120', '125', '175'],
 3: ['130', '133', '135'],
 4: ['140', '145', '160'],
 5: ['150', '155', '160'],
 6: ['156', '183'],
 7: ['184', '233'],
 8: ['163', '180'],
 9: ['170', '190', '217']}